In [ ]:
import sys
from pathlib import Path

# Add the 'backtest' directory to the system path
notebook_dir = Path().resolve()
backtest_dir = notebook_dir.parent / 'backtest'
sys.path.append(str(backtest_dir))

In [ ]:
import pandas as pd
from backtest import Backtest, Strategy, TradeAction

In [ ]:
# First run the get_data.ipynb notebook to generate the data file
# or use the code from get_data.ipynb here to download the data directly

# Read the OHLCV data from data/BTCUSDT.csv
# Use small subset of data for easier analysis
data = pd.read_csv('data/BTCUSDT.csv', index_col='Date', parse_dates=True)

In [ ]:
class MeanReversionStrategy(Strategy):
    def __init__(self):
        self.sma_period = 50
        self.zscore_threshold = 2
    
    def on_candle(self, historical_data, positions_list):
        df = historical_data.copy()

        # not enough data
        if(len(df) < self.sma_period):
            return []
        
        # Calculate the simple moving average
        df.loc[:, 'sma'] = df['Close'].rolling(window=self.sma_period).mean()
        df.loc[:, 'stdev'] = df['Close'].rolling(window=self.sma_period).std()

        # Check if for some reason the last stdev is NaN or 0
        if pd.isna(df.iloc[-1]['stdev']) or df.iloc[-1]['stdev'] == 0:
            return []

        zScore = (df.iloc[-1]['Close'] - df.iloc[-1]['sma']) / df.iloc[-1]['stdev']

        # I should really start sending only open positions to the strategy
        # but I will probably remove multiple positions completely anyway
        open_positions = [pos for pos in positions_list if pos.exit_time is None]

        if(zScore > self.zscore_threshold):
            # Check if we have an open position
            if self.has_short_positions(open_positions):
                return []
            # Otherwise close all open positions and open a new short position
            return [TradeAction(action="enter", quantity=-0.5)] + [
                TradeAction(action="exit", position_id=pos.id) for pos in open_positions
            ]
        elif(zScore < -self.zscore_threshold):
            # Check if we have an open position
            if self.has_long_positions(open_positions):
                return []
            # Otherwise close all positions and open a new long position
            return [TradeAction(action="enter", quantity=0.5)] + [
                TradeAction(action="exit", position_id=pos.id) for pos in open_positions
            ]
        # No action
        return []
    
    def has_short_positions(self, open_positions):
        return any([pos for pos in open_positions if pos.quantity < 0])
    
    def has_long_positions(self, open_positions):
        return any([pos for pos in open_positions if pos.quantity > 0])

In [ ]:
backtest = Backtest(data, MeanReversionStrategy, equity=100000.0)

backtest.run()

backtest.pnl_df

In [ ]:
stats = backtest.stats()

In [ ]:
# Extract long positions (quantity >= 0 or value >= 0)
long_positions = [pos for pos in backtest.positions if pos.quantity >= 0]
# Create a series of long positions entry "Open" prices from backtest.pnl_df
long_entry_series = backtest.pnl_df.loc[[pos.entry_time for pos in long_positions], 'Open']

# Extract short positions (quantity < 0 or value < 0)
short_positions = [pos for pos in backtest.positions if pos.quantity < 0]
# Create a series of short position entry "Open" prices from backtest.pnl_df
short_entry_series = backtest.pnl_df.loc[[pos.entry_time for pos in short_positions], 'Open']

In [ ]:
from matplotlib import pyplot as plt

df = backtest.pnl_df

plt.figure(figsize=(14, 6))
plt.plot(df['Open'], label="Open Price", alpha=0.7)

plt.scatter(long_entry_series.index, long_entry_series, marker='^', color="green", label="Long", s=100)

plt.scatter(short_entry_series.index, short_entry_series, marker='v', color="red", label="Short", s=100)

for idx, price in long_entry_series.items():
    plt.annotate(idx.strftime('%Y-%m-%d'), xy=(idx, price), xytext=(0, 10), 
                 textcoords='offset points', ha='center', fontsize=8, color='green')
    
for idx, price in short_entry_series.items():
    plt.annotate(idx.strftime('%Y-%m-%d'), xy=(idx, price), xytext=(0, -15), 
                 textcoords='offset points', ha='center', fontsize=8, color='red')

plt.title("BTC/USDT with Long/Short Entry Points")
plt.xlabel('Date')
plt.ylabel('Price (USDT)')
plt.legend()
plt.grid(True)
plt.show()

# Plot the Total PnL
plt.figure(figsize=(14, 2))
plt.plot(df['total_pnl'], label="Total PnL")
plt.title("Total PnL Over Time")
plt.xlabel('Date')
plt.ylabel('Total PnL (USDT)')
plt.legend()
plt.grid(True)
plt.show()

# Plot the Z-Score
df['sma'] = df['Close'].rolling(backtest.strategy.sma_period).mean()
df['stdev'] = df['Close'].rolling(backtest.strategy.sma_period).std()

df['zScore'] = (df['Close'] - df['sma']) / df['Close'].rolling(backtest.strategy.sma_period).std()
df['zScore'] = df['zScore'].fillna(0)

plt.figure(figsize=(14, 2))
plt.plot(df['zScore'], label="Z-Score")
plt.title("Z-Score Over Time")
plt.xlabel('Date')
plt.ylabel('Z-Score')
plt.legend()
plt.grid(True)
plt.show()